<a href="https://colab.research.google.com/github/asmijuneja/AI-ML-Project-Series/blob/main/Project_3_Disease_Prediction_System_using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

training_data = pd.read_csv('Training.csv')

uploaded = files.upload()

testing_data = pd.read_csv('Testing.csv')


print(training_data.head())
print(testing_data.head())

Saving Training.csv to Training.csv
Saving Testing.csv to Testing.csv


   itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   
3        1          1                     0                    0          0   
4        1          1                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  scurring  \
0       0           0             0        0                 0  ...         0   
1       0           0             0        0                 0  ...         0   
2       0           0             0        0                 0  ...         0   
3       0           0             0        0                 0  ...         0   
4       0           0             0        0                 0  ...         0   

   skin_peeling  silver_like_dusting  

In [2]:
training_data = training_data.loc[:, ~training_data.columns.str.contains('^Unnamed')]
testing_data = testing_data.loc[:, ~testing_data.columns.str.contains('^Unnamed')]

print(training_data.head())
print(testing_data.head())

print(f"Missing values in training data: {training_data.isnull().sum().sum()}")
print(f"Missing values in testing data: {testing_data.isnull().sum().sum()}")


   itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   
3        1          1                     0                    0          0   
4        1          1                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  \
0       0           0             0        0                 0  ...   
1       0           0             0        0                 0  ...   
2       0           0             0        0                 0  ...   
3       0           0             0        0                 0  ...   
4       0           0             0        0                 0  ...   

   blackheads  scurring  skin_peeling  silver_like_dusting  \
0           0         0             

In [3]:
X_train = training_data.drop('prognosis', axis=1)
y_train = training_data['prognosis']
X_test = testing_data.drop('prognosis', axis=1)
y_test = testing_data['prognosis']


In [4]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


from sklearn.tree import DecisionTreeClassifier


model = DecisionTreeClassifier(random_state=42)

model.fit(X_train_scaled, y_train_encoded)

DecisionTreeClassifier(random_state=42)

In [6]:

new_predictions = model.predict(X_test_scaled[:5])
print("Predicted Prognosis:", le.inverse_transform(new_predictions))

Predicted Prognosis: ['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction']


In [7]:

from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X_train_scaled, y_train_encoded, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")

Cross-Validation Scores: [1. 1. 1. 1. 1.]


In [8]:

from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train_encoded)
print(f"Best Parameters: {grid_search.best_params_}")

Best Parameters: {'max_depth': None, 'min_samples_split': 2}


In [9]:

feature_importances = model.feature_importances_
important_features = pd.Series(feature_importances, index=X_train.columns).sort_values(ascending=False)
print(f"Feature Importances:\n{important_features}")


Feature Importances:
high_fever                      0.050272
muscle_pain                     0.025000
enlarged_thyroid                0.025000
malaise                         0.025000
mild_fever                      0.025000
                                  ...   
throat_irritation               0.000000
phlegm                          0.000000
blurred_and_distorted_vision    0.000000
swelled_lymph_nodes             0.000000
bruising                        0.000000
Length: 132, dtype: float64


In [10]:
import pickle

# Save the Decision Tree model
with open('decision_tree_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the Label Encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# Save the Scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test_encoded, y_pred)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test_encoded, y_pred)

# Generate a detailed classification report
class_report = classification_report(y_test_encoded, y_pred, target_names=le.classes_)


print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.9761904761904762
Confusion Matrix:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Classification Report:
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1

In [12]:

new_predictions = model.predict(X_test_scaled[:5])
print("Predicted Prognosis:", le.inverse_transform(new_predictions))


Predicted Prognosis: ['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction']
